In [1]:
 !wget "https://raw.githubusercontent.com/viraj-lakshitha/animal-disease-symptom-ontology/develop/ADSOv1.0.3.owl";

--2023-01-05 09:53:19--  https://raw.githubusercontent.com/viraj-lakshitha/animal-disease-symptom-ontology/develop/ADSOv1.0.3.owl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68571 (67K) [text/plain]
Saving to: ‘ADSOv1.0.3.owl’

ADSOv1.0.3.owl      100%[===================>]  66.96K  --.-KB/s    in 0.001s  

2023-01-05 09:53:19 (54.4 MB/s) - ‘ADSOv1.0.3.owl’ saved [68571/68571]



In [2]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 859.6 kB/s eta 0:00:0000:01


In [15]:
!pip install pprintpp

In [3]:
%ls

ADSOv1.0.3.owl  LICENSE  README.md  data-loader.ipynb


In [1]:
from rdflib import Graph, Namespace, Literal, RDF, URIRef

g = Graph()

In [2]:
list(g.namespaces())

[('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace'))]

In [3]:
g.parse("ADSOv1.0.3.owl")

<Graph identifier=N4ba74ec67964439eb22842c7b38ceb98 (<class 'rdflib.graph.Graph'>)>

In [4]:
len(g)

746

In [5]:
from rdflib.namespace import OWL

for idx,relation in enumerate(g):
    if idx == 5: # Print first five relations
        break
    print(relation, "\n")

(rdflib.term.URIRef('https://ontology.drpawspaw.com/ADSO0000000048'), rdflib.term.URIRef('https://ontology.drpawspaw.com/text'), rdflib.term.Literal('Swelling Limbs', lang='en')) 

(rdflib.term.URIRef('https://ontology.drpawspaw.com/ADSO0000000090'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#NamedIndividual')) 

(rdflib.term.URIRef('https://ontology.drpawspaw.com/ADSO0000000096'), rdflib.term.URIRef('https://ontology.drpawspaw.com/text'), rdflib.term.Literal('Discharge from the Valva', lang='en')) 

(rdflib.term.URIRef('https://ontology.drpawspaw.com/ADSO0000000108'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('walking_problems', lang='en')) 

(rdflib.term.URIRef('https://ontology.drpawspaw.com/ADSO0000000026'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://ontology.drpawspaw.com/symptom')) 



In [6]:
# Get the URIRef for given keyword
def get_ref(keyword):
    return URIRef("https://ontology.drpawspaw.com/"+keyword)

In [7]:
# Get the entity 'text' from the URIRef
def get_text_from_uri(uri):
    for s,p,o in g:
        if s == uri and p == URIRef("https://ontology.drpawspaw.com/text"):
            return o

In [8]:
# Collect named entities from ontology
diseases = []
symptoms = []
synonyms = []

for s,p,o in g:
    if p == get_ref("hasDisease"):
        try:
            dis = get_text_from_uri(o).toPython()
            diseases.append(dis)
        except:
            print(type(o), o)
    if p == get_ref("hasSymptom"):
        try:
            sym = get_text_from_uri(o).toPython()
            symptoms.append(sym)
        except:
            print(type(o), o)
    if p == get_ref("hasSynonym"):
        try:
            syn = get_text_from_uri(o).toPython()
            synonyms.append(syn)
        except:
            print(type(o), o)

<class 'rdflib.term.URIRef'> https://ontology.drpawspaw.com/ADSO0000000110


In [9]:
diseases

['Distempter',
 'Pneumonia',
 'IMHA',
 'Parvo',
 'Babesiosis',
 'Ear Infections',
 'Bone Fractures',
 'Pyometra',
 'Idiopathic Seizures',
 'Demodicosis']

In [10]:
get_text_from_uri(URIRef("https://ontology.drpawspaw.com/ADSO0000000110"))

In [11]:
entities = diseases+synonyms+symptoms

In [18]:
# List of symptoms and their synonyms
symp_syn = []

def is_exist(keyword, collection):
    for _,k,_ in collection:
        if k == keyword:
            return True

for s,p,o in g:
    # Filter synonyms
    if p == get_ref("hasSymptom"):
        try:
            x = get_text_from_uri(o).toPython()
        except:
            continue

        curr_symp_syn = []
        for s1, p1, o1 in g:
            # Filter synonym for above "o" entity
            if s1 == o and p1 == get_ref("hasSynonym"):
                try:
                    y = get_text_from_uri(o1).toPython()
                    curr_symp_syn.append(y)                    
                except:
                    continue
        # if len(curr_symp_syn) != 0 and not is_exist(x ,symp_syn):
        if not is_exist(x ,symp_syn):
            try:
                idx = o.toPython()
                symp_syn.append((idx, x, curr_symp_syn))
            except:
                continue

In [20]:
symp_syn

[('https://ontology.drpawspaw.com/ADSO0000000059', 'Bloated Tummy', []),
 ('https://ontology.drpawspaw.com/ADSO0000000065', 'Unusual Collapse', []),
 ('https://ontology.drpawspaw.com/ADSO0000000048', 'Swelling Limbs', []),
 ('https://ontology.drpawspaw.com/ADSO0000000007', 'coughing', []),
 ('https://ontology.drpawspaw.com/ADSO0000000045', 'Weight Loss', []),
 ('https://ontology.drpawspaw.com/ADSO0000000027', 'Eye Discharge', []),
 ('https://ontology.drpawspaw.com/ADSO0000000008', 'Vomiting', []),
 ('https://ontology.drpawspaw.com/ADSO0000000078', 'Thickening of Skin', []),
 ('https://ontology.drpawspaw.com/ADSO0000000005', 'Fever', []),
 ('https://ontology.drpawspaw.com/ADSO0000000034', 'Muscle Twitching', []),
 ('https://ontology.drpawspaw.com/ADSO0000000057', 'Nausea', []),
 ('https://ontology.drpawspaw.com/ADSO0000000016', 'Blood Diarrhea', []),
 ('https://ontology.drpawspaw.com/ADSO0000000070', 'Kidney Disease', []),
 ('https://ontology.drpawspaw.com/ADSO0000000082', 'Fast Breathi

In [55]:
%store symp_syn

Stored 'symp_syn' (list)
